# **Clustering Assignment 2: Sessa Empirical Estimator (SEE)**
## 📝 **Instructions**
1) **Read the Journals** about the Sessa Empirical Estimator.
2) **Convert the R codes into Python** Codes (use jupyter notebook).
3) **Perform the Sessa Empircal Estimator and generate some insights** using either: 
    - Simulated data (https://www.frontiersin.org/journals/pharmacology/articles/10.3389/fphar.2019.00383/full) or 
    - Real world datasets of your choice (You can obtain it in Kaggle or in https://archive.ics.uci.edu/),
4) **Generate a new insight using the new clustering algorithm**
    - SEE uses K-Means clustering (again recall the disadvantages of K-Means)
    - Substitute a different clustering algorithm
5) **Compare your results between**:
    - SEE using K-Means, and 
    - SEE using the clustering algorithm of your choice.
6) Deadline is this **Sunday, Feb 23,** ~~2022~~ **2025 at 11:59 pm**
7) Do this with your thesis partner.
8) You can use any A.I. assistant.
---

## 👩‍💻 **Converting R Code to Python Code**
### 💥**Importing necessary libraries** 💥
#### Data Handling
1. `pandas`: Used for handling tabular data (like data frames in R). Equivalent to `dplyr` in R
2. `numpy`: Provides numerical operations, including array manipulations and mathematical functions.

In [1]:
import pandas as pd
import numpy as np

#### Data Visualization
1.  `matplotlib.pyplot`: Equivalent to R's `plot()`, used for static graphs.
2. `seaborn`: Used for statistical visualizations, like box plots (similar to R's `ggplot2`).


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

#### Statistical Distribution
1. `gaussian_kde` (Kernel Density Estimation): Similar to R's `density()`, used to estimate the probability distribution of a continuous variable.

In [3]:
from scipy.stats import gaussian_kde

### Machine Learning (Clustering)
1. `KMeans`: Implements K-Means clustering (equivalent to R’s kmeans()).
2. `DBSCAN`: Alternative for K-Means clustering for SEE
3. `StandardScaler`: Standardizes data (similar to R's scale()).

In [5]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

#### **Summary Table of R-to-Python Imports**
| Functionality | R Equivalent | Python Equivalent |
| --- | --- | --- |
| Data Manipulation | `dplyr`, `data.table` | `pandas` |
| Numerical Computations | `base::log`, `scale()` | `numpy`, `scipy` |
| Plotting | `ggplot2`, `plot()`	| `seaborn`, `matplotlib.pyplot` |
| Kernel Density Estimation | `density()` | `gaussian_kde` |
| Clustering (K-Means) | `kmeans()` | `sklearn.cluster.KMeans` |
| Clustering (DBSCAN) | *-* | `sklearn.cluster.DBSCAN` | 
| Data Scaling | `scale()` | `sklearn.preprocessing.StandardScaler` |


### 💥**Main Functions**💥
1. `see(arg1, tidy)` – Implements the **Sessa Empirical Estimator (SEE)** using K-Means and DBSCAN (alt) clustering.

In [8]:
def see(arg1, tidy, clustering_algo):
    # Filtering the dataset based on a given drug (arg1).
    #   Selects only rows where the ATC (drug code) matches arg1.
    C09CA01 = tidy[tidy['ATC'] == arg1]
    #   Makes a copy of the filtered data.
    Drug_see_p1 = C09CA01.copy()

    # Compute Event Intervals
    #   Sorts by patient ID (pnr) and prescription date (eksd).
    Drug_see_p1 = Drug_see_p1.sort_values(by=['pnr', 'eksd'])
    #   Uses shift(1) to get the previous prescription date per patient.
    Drug_see_p1['prev_eksd'] = Drug_see_p1.groupby('pnr')['eksd'].shift(1)
    #   Drops NaN values (first prescription has no previous date).
    Drug_see_p1 = Drug_see_p1.dropna()
    #   Computes the difference (event interval) in days.
    Drug_see_p1['event_interval'] = (Drug_see_p1['eksd'] - Drug_see_p1['prev_eksd']).dt.days
    
    # Generate Empirical CDF
    #   Creates ECDF values (x, y), similar to R’s ecdfplot().
    x = np.sort(Drug_see_p1['event_interval'].values)
    y = np.arange(1, len(x) + 1) / len(x)
    dfper = pd.DataFrame({'x': x, 'y': y})
    #   Filters the lower 80% of the ECDF to remove outliers.
    dfper = dfper[dfper['y'] <= 0.8]
    
    # Plot ECDF
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(dfper['x'], dfper['y'], label='80% ECDF')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(x, y, label='100% ECDF')
    plt.legend()
    plt.show()
    
    # Density Estimation
    #   Computes the log-transformed density estimation (like R’s density()).
    log_event_intervals = np.log(dfper['x'])
    #   Estimate probability distribution.
    density = gaussian_kde(log_event_intervals)
    x_vals = np.linspace(min(log_event_intervals), max(log_event_intervals), 1000)
    y_vals = density(x_vals)
    
    plt.figure(figsize=(6, 4))
    plt.plot(np.exp(x_vals), y_vals)
    plt.title("Log(Event Interval) Density")
    plt.show()
    
    # Clustering
    if clustering_algo == 'dbscan':
    #   Alternate clustering (DBSCAN)
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(dfper[['x']])
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        dfper['cluster'] = dbscan.fit_predict(scaled_data)
    else:
    #   Default clustering (K-Means)
        #   Standardizes event intervals
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(dfper[['x']])
        silhouette_scores = []
        for k in range(2, 10):
            kmeans = KMeans(n_clusters=k, random_state=1234)
            kmeans.fit(scaled_data)
            silhouette_scores.append(kmeans.inertia_)
        #   Finds the optimal number of clusters (best_k) using K-Means inertia.
        best_k = np.argmax(silhouette_scores) + 2
        #    Assigns each event interval to a K-Means cluster.
        kmeans = KMeans(n_clusters=best_k, random_state=1234)
        dfper['cluster'] = kmeans.fit_predict(scaled_data)
    
    return dfper

2. `see_assumption(tidy)` – Analyzes the assumptions behind the SEE, using **box plots and distribution analysis**.

In [7]:
def see_assumption(tidy):
    # Compute event durations
    #   Sorts prescriptions per patient.
    tidy = tidy.sort_values(by=['pnr', 'eksd'])
    tidy['prev_eksd'] = tidy.groupby('pnr')['eksd'].shift(1)
    tidy = tidy.dropna()
    #   Computes the days between consecutive prescriptions.
    tidy['Duration'] = (tidy['eksd'] - tidy['prev_eksd']).dt.days
    
    # Generate boxplots
    # Helps identify anomalies in prescription patterns.
    plt.figure(figsize=(8, 4))
    sns.boxplot(x='pnr', y='Duration', data=tidy)
    plt.xticks(rotation=90)
    plt.show()
    
    return tidy[['pnr', 'Duration']]

# Example usage:
# tidy['eksd'] = pd.to_datetime(tidy['eksd'])
# result_A = see('medA', tidy)
# result_B = see('medB', tidy)
# see_assumption(result_A)


### **Final Summary of Function Roles**
| **Function**       | **Purpose** |
|--------------------|------------|
| `see(arg1, tidy)` | Implements **Sessa Empirical Estimator (SEE)** using **K-Means and DBSCAN clustering** on prescription intervals. |
| `see_assumption(tidy)` | Analyzes **intervals between prescriptions** using **box plots and distribution analysis**. |


---
## 👨‍🏭 **Perform SEE on Simulated Data**